In [1]:
from glob import glob
import xarray as xr
import cf_xarray # use cf-xarray so that we can use CF attributes
import pandas as pd
import matplotlib.pyplot as plt
import regionmask
import cartopy.crs as ccrs
import nc_time_axis
import numpy as np
import warnings
# To access collection
import dask
import intake
# from distributed import Client
from dask_jobqueue import PBSCluster
from dask.distributed import Client, LocalCluster, futures_of
from dask.diagnostics import ProgressBar
# from dask.distributed import progress
from tqdm import tqdm 
import regionmask
from xclim.indices.stats import frequency_analysis


# from ncar_jobqueue import NCARCluster
# https://ncar.github.io/esds/faq/#xarray-and-dask

# USE SCIENCEPLOTS

Following this tutorial --> https://ncar.github.io/esds/posts/2021/intake-cesm2-le-glade-example/

also --> https://ncar.github.io/esds/posts/2021/intake_esm_dask/

## Spin up Dask cluster

Useful thread --> https://github.com/dask/dask-jobqueue/issues/216

Also --> https://github.com/NCAR/ncar-jobqueue/issues/101

https://github.com/NCAR/dask-tutorial/blob/main/notebooks/05-dask-hpc.ipynb

Ncar yaml located at `~/.config/dask` --> changed interface to `ext`

In [3]:
# Create our NCAR Cluster - which uses PBSCluster under the hood
num_jobs = 1
cluster = PBSCluster(
    job_name = 'valencig-dask-hpc',
    cores = 1,
    memory = '10GiB',
    processes = 1,
    local_directory = '/glade/u/home/valencig/spilled/',
    log_directory = '/glade/u/home/valencig/worker-logs/',
    resource_spec = 'select=1:ncpus=1:mem=15GB',
    queue = 'casper',
    walltime = '02:00:00', # Change wall time if needed
    interface = 'ext'
)


# Spin up workers
cluster.scale(jobs=num_jobs)

# Assign the cluster to our Client
client = Client(cluster)

# Block progress until workers have spawned
client.wait_for_workers(num_jobs)

In [3]:
client

<Client: 'tcp://128.117.208.82:34383' processes=6 threads=6, memory=60.00 GiB>

In [4]:
cluster.get_logs()

{'Cluster': '',
 'Scheduler': "2023-12-14 09:34:22,677 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.76:44309\n2023-12-14 09:34:22,677 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.76:44309', name: PBSCluster-0, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:22,676 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.76:45411\n2023-12-14 09:34:22,676 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.76:45411', name: PBSCluster-5, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:22,676 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.76:42007\n2023-12-14 09:34:22,676 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.76:42007', name: PBSCluster-6, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:22,676 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.76:40813\n2023-12-14 09:34:22,675 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.76:40813', name: PBSCluster-1, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:21,349 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.78:44371\n2023-12-14 09:34:21,349 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.78:44371', name: PBSCluster-2, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:20,535 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.57:40335\n2023-12-14 09:34:20,535 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.57:40335', name: PBSCluster-3, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:20,507 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.75:38321\n2023-12-14 09:34:20,507 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.75:38321', name: PBSCluster-9, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:20,506 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.75:42323\n2023-12-14 09:34:20,506 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.75:42323', name: PBSCluster-8, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:19,024 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.84:43013\n2023-12-14 09:34:19,024 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.84:43013', name: PBSCluster-7, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:19,023 - distributed.scheduler - INFO - Starting worker compute stream, tcp://128.117.208.84:42703\n2023-12-14 09:34:19,023 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://128.117.208.84:42703', name: PBSCluster-4, status: running, memory: 0, processing: 0>\n2023-12-14 09:34:11,924 - distributed.scheduler - INFO - Receive client connection: Client-9c285a70-9a9e-11ee-96b4-3cecef1b12c2\n2023-12-14 09:34:11,042 - distributed.scheduler - INFO - Registering Worker plugin shuffle\n2023-12-14 09:34:11,042 - distributed.scheduler - INFO -   dashboard at:  https://jupyterhub.hpc.ucar.edu/stable/user/valencig/proxy/8787/status\n2023-12-14 09:34:11,041 - distributed.scheduler - INFO -   Scheduler at: tcp://128.117.208.82:34383\n2023-12-14 09:34:10,923 - distributed.scheduler - INFO - State start",
 'tcp://128.117.208.57:40335': '2023-12-14 09:34:20,536 - distributed.worker - INFO - -------------------------------------------------\n2023-12-14 09:34:20,536 - distributed.worker - INFO -         Registered to: tcp://128.117.208.82:34383\n2023-12-14 09:34:20,535 - distributed.worker - INFO - Starting Worker plugin shuffle\n2023-12-14 09:34:20,527 - distributed.worker - INFO - -------------------------------------------------\n2023-12-14 09:34:20,527 - d

### Commands for managing dask workers

https://arc.ucar.edu/knowledge_base/68878389

In [ ]:
# See the workers in the job scheduler
!qstat -u $USER

# Kill all running or pending jobs
# !qdel `qselect -u $USER`

## Read in the catalog

In [5]:
# 'cesm.json' is copy of '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
# Comment out "options": null in aggregation_controls.aggregations.0 in order to get intake-esm to work
# cat = intake.open_esm_datastore('cesm2.json')
cat = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json')
cat

,unique
activity_id,17
institution_id,35
source_id,81
experiment_id,130
member_id,421
table_id,36
variable_id,1075
grid_label,12
dcpp_init_year,59
version,597


## Querying for desired variable

https://www.cesm.ucar.edu/community-projects/lens/data-sets

CMIP6 variable list --> https://na-cordex.org/variable-list.html

Also --> https://wcrp-cmip.github.io/CMIP6_CVs/docs/CMIP6_experiment_id.html

In [6]:
# cat.search(component='atm', long_name=['wind*', 'Wind*']).df.long_name.unique()
cat.search(variable_id=['wind*', 'Wind*']).df.variable_id.unique()

array(['sfcWind', 'sfcWindmax', 'sndmasswindrif'], dtype=object)

## Query and subset data catalog

Overview found [here](https://www2.cesm.ucar.edu/projects/CMIP6/):

ScenarioMIP: "Will provide multi-model climate projections based on alternative scenarios of future emissions and land use changes produced with integrated assessment models. The design consists of eight alternative 21st century scenarios plus one large initial condition ensemble and a set of long-term extensions. Climate model projections will facilitate integrated studies of climate change as well as address targeted scientific questions."

Citation: O'Neill, B. C., Tebaldi, C., van Vuuren, D.P., Eyring, V., Friedlingstein, P., Hurtt, G., Knutti, R., Kriegler, E., Lamarque, J.-F., Lowe, J., Meehl, G.A., Moss, R., Riahi, K., and Sanderson, B. M. 2016. The Scenario Model Intercomparison Project (ScenarioMIP) for CMIP6. Geosci. Model Dev., 9, 3461-3482.

In [7]:
cesm2 = cat.search(
    variable_id='sfcWind', # near surface wind
    source_id='CESM2',
    experiment_id='ssp*',
    # experiment_id='historical', # all historical forcings
    table_id='day', # day is highest resolution
    activity_id='ScenarioMIP'
)

In [8]:
cesm2.keys_info()

,activity_id,institution_id,source_id,experiment_id,table_id,grid_label
key,,,,,,
ScenarioMIP.NCAR.CESM2.ssp126.day.gn,ScenarioMIP,NCAR,CESM2,ssp126,day,gn
ScenarioMIP.NCAR.CESM2.ssp245.day.gn,ScenarioMIP,NCAR,CESM2,ssp245,day,gn
ScenarioMIP.NCAR.CESM2.ssp370.day.gn,ScenarioMIP,NCAR,CESM2,ssp370,day,gn
ScenarioMIP.NCAR.CESM2.ssp585.day.gn,ScenarioMIP,NCAR,CESM2,ssp585,day,gn


## Read in using ```.to_dataset_dict()```

https://stackoverflow.com/questions/67813208/xarray-open-mfdataset-doesnt-work-if-dask-distributed-client-has-been-created

In [9]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets = cesm2.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Let’s take a look at the keys - these are defined by the `groupby` attributes in the catalog. The groupby attributes in this case are:

https://www2.cesm.ucar.edu/experiments/cesm1.2/GLE/GLENS_output_fields/

`component.experiment.stream.forcing_variant.control_branch_year.variable`

- Component - which component this output is from (ex. atm represents the atmosphere)
- Experiment - which experiment this is from, in this case, this is `ssp370` which is one of the CMIP6 future experiments
- Stream - which stream this output is from, in this case, this is `cam.h1`, which represents daily output
- Control Branch Year - which year the ensemble branched off from, these are described within the [CESM2-LE documentation page](https://www.cesm.ucar.edu/community-projects/lens2)
- Variable - which variable you are working with


component = atm (atmosphere), lnd (land), ocn (ocean), ice

frequency = monthly, daily, or hourly6

experiment = historical (1850 to 2015) or ssp370 (2015 to 2100)

forcing_variant = the biomass forcing variant, cmip6 (the default in the cmip6 runs) or smbb (smoothed biomass burning)

variable = one of the variable names listed in the tables below

## Process Data

Like Shen (2022) we only take the first model member.

"Note that only the first ensemble member (index of “r1i1p1f1”) is selected from each model"

For extreme wind speeds, refer to [this site](https://www.wind-pioneers.com/extreme-wind-analysis-at-a-wind-turbine-site/). If the Weibull distribution shape factor $k$ can be determined then the [EWTS II](https://openwind.ul-renewables.com/ewts.html) algorithm works.

In [16]:
# !!! SWITCH BACK TO 2100 WHEN USING SSP DATA !!!

def subset_ds(ds, task):
    states = regionmask.defined_regions.natural_earth_v5_0_0.us_states_50
    # Hawaii and Alaska are not included in the mask
    good_keys = [
        k for k in states.regions.keys() 
        if k not in states.map_keys(['Hawaii', 'Alaska'])
    ]
    mask = states.mask(ds.lon, ds.lat).isin(good_keys)
    da = ds.where(mask, drop=True).sfcWind.isel(member_id=0).sel(time=slice('1978', '2100')) # Last time (2100) is wonky
    if task == 'mean':
        result = da.resample(time='1Y').mean(dim='time')
    elif task == 'anomaly':
        # Anomaly is x-x_mean 
        # Then get average anomaly on a yearly basis
        result = (da - da.mean('time')).resample(time='1Y').mean('time')
    return result

In [17]:
# ISSUE: NEED TO MASK TO ONLY BE LAND DATA
task = 'mean'
print(f'Running task: {task}')
for key in tqdm(list(dsets.keys()), desc='Processing Data...'):
    ds = dsets[key].chunk({'time':365}) # Get into approx 100 mb chunks
    da = subset_ds(ds, task=task).persist()
    da.compute().to_netcdf('/glade/u/home/valencig/wind-trend-analysis/data/'+key+'.'+task+'.nc')

Running task: mean


Processing Data...:   0%|          | 0/4 [00:15<?, ?it/s]


## Restart dask cluster

In [ ]:
client.restart()

## Close dask cluster

In [4]:
client.shutdown()